Compare pycortex and freesurfer surface projections

In [1]:
import cortex
import cortex.surfinfo
import os, sys, shutil
import numpy as np
import nibabel as nib
subject='Danny' # 'bert'

In [2]:
fsp = '/Users/chris/Dropbox/fsSYNC/subjects/' + subject
mmp = '/Users/chris/Dropbox/GIT_Support/NHP-BIDS/manual-masks/sub-' + subject.lower() + '/func'

In [5]:
cortex.freesurfer.import_subjnhp(subject)

Are you sure you want to overwrite this existing subject?
This will delete all files for this subject in the filestore, including all blender cuts. Type YES
 yes


ValueError: Do not overwrite

In [6]:
# copy FS flatmaps to expected naming scheme
hemi = ['lh','rh']
for xh in hemi:
    src_file = fsp + '/surf/' + xh + ".full.patch.flat"
    trg_file = fsp + '/surf/' + xh + ".full.flat.patch.3d"
    shutil.copyfile(src_file,trg_file)

In [7]:
cortex.freesurfer.import_flat(subject,"full")

flatmap vertex locations for this subject, and will result
in deletion of the overlays.svg file and all cached info
for this subject (because flatmaps will fundamentally change).
Proceed? [y]/n:  y


b'created by chris on Tue Mar  5 13:00:01 2019\n'
saving to /Users/chris/Dropbox/CURRENT_PROJECTS/NHP_MRI/pycortex/db/Danny/surfaces/flat_lh.gii
b'created by chris on Tue Mar  5 13:33:56 2019\n'
saving to /Users/chris/Dropbox/CURRENT_PROJECTS/NHP_MRI/pycortex/db/Danny/surfaces/flat_rh.gii
No overlays.svg file to delete


In [58]:
# convert the existing transform from functional to FS-anatomical into a pycortex format
xfm = "fullhead"

# copy anatomy to expected filename
shutil.copyfile(fsp + '/mri/T1.mgz',fsp + '/mri/orig.mgz')

# the fsl-file from flirt or the tkreg file from freesurfer
#transf = fsp + '/manual-masks_toFS/func/epi2anat.mat'
transf = fsp + '/manual-masks_toFS/func/reg.fsl.dat'

# functional
func = mmp + '/sub-' + subject.lower() + '_ref_func_res-1x1x1.nii.gz'

# FS anatomical 
anat = fsp + '/mri/brainmask.mgz'

# convert
#res=cortex.xfm.Transform.from_fsl(transf,func,anat).save('Danny',xfm)
xfmmat=cortex.xfm.Transform.from_freesurfer(transf,func,subject)
xfmmat.save(subject,xfm)
#xfmmat.to_freesurfer(subject,'/User/chris/Desktop')

KeyError: '/User/chris/Desktop'

In [59]:
xfm = "fullhead"
#xfm="example-transform"
test_data = '/Users/chris/Desktop/R2.nii.gz'

In [60]:
dv = cortex.Volume(test_data, subject, xfm)
cortex.webshow(dv)

Started server on port 61711


<JS: window.viewer>

Stopping server
Stopping server


In [29]:
dv = cortex.Volume(test_data, subject, xfm)
cortex.webshow(dv,title='NHP')

ValueError: Volumetric data (shape (256, 256, 256)) is not the same shape as reference for transform (shape (56, 90, 90))

In [42]:
# try to apply the pycortex transfer and see if alignment is correct
epi = cortex.Volume(func, subject, xfm)
dv2 = cortex.volume.epi2anatspace(epi)
cortex.webshow(dv2)

TypeError: Unknown input type

In [ ]:
dv = cortex.Volume(test_data, subject, xfm)

In [23]:
import subprocess as subproc
subproc.check_call(['mri_vol2surf','--trgsubject',subject,'--src',test_data,'--out','/Users/chris/Desktop/lh.test.w', 
                 '--out_type','paint','--projfrac','0.7','--srcreg',transf,'--hemi', 'lh'])

0

Stopping server


When using the tkReg file in mri_vol2surf, the reulting 'paintt files are aligned with the surfaces

In [30]:
subproc.check_call(['mri_vol2vol',
                    '--reg','/Users/chris/Dropbox/fsSYNC/subjects/Danny/manual-masks_toFS/func/reg.fsl.dat',
                    '--mov','/Users/chris/Desktop/R2.nii.gz', 
                    '--s',subject,
                    '--o', '/Users/chris/Desktop/test2anat.nii.gz']
                    )

CalledProcessError: Command '['mri_vol2vol', '--reg', '/Users/chris/Dropbox/fsSYNC/subjects/Danny/manual-masks_toFS/func/reg.fsl.dat', '--mov', '/Users/chris/Desktop/R2.nii.gz', '--s', 'Danny', '--o', '/Users/chris/Desktop/test2anat.nii.gz']' returned non-zero exit status 1.

When applying the tkReg registration to test-data (results) they end up aligned with the orig.mgz volume
So this registration is fine